## Extraction de données d'un fichier pour automatiser leur traitement
Le but de cet exercice est de travailler sur l'extraction de données à partir de fichiers. Le sujet étant très vaste, on va ici étudier certaines méthodes et leurs alternatives. Si python permet de faire tout en une seule étape, pour des très gros fichiers (de l'ordre de la centaine de méga-octets), il se peut que cela soit trop lent et qu'il faille lui préférer des outils en ligne de commande, un des plus usités étant `awk`.


On cherche à analyser un fichier généré par un programme de chimie quantique. Celui-ci contient un très grand nombre d'informations et on cherche à en extraire certaines informations. Entre autre, on va chercher à extraire :
- des symétries d'orbitales qui sont listées sur plusieurs lignes consécutives
- la projection d'une orbitale moléculaire sur des orbitales atomiques : cela correspond à des blocs de textes similaires qui se répètent.

Le fichier à analyser s'appelle `exobonus-alizarine.log`, il fait 4018 lignes et a été généré avec le logiciel gaussian. Bien que l'exemple soit spécifique, la succession d'actions est relativement générique et pourra être adaptée à d'autres cas moyennant quelques changements. Le fichier est ici directement au format texte, ce qui simplifie les vérifications. Mais il se peut que le format d'origine soit plus complexe comme un fichier de type hdf ou un fichier binaire.

Globalement, on va avoir une succession de blocs qui vont décrire certains résultats. Puis à l'intérieur d'un bloc, on va pouvoir avoir la répétition de sous-bloc identiques que l'on cherche à analyser.

![pics/struc-files.svg](pics/struc-files.svg)

# Objectifs
On va chercher à :
* récupérer les symétries des orbitales qui sont données dans un bloc de text
* récupérer des coefficients d'orbitales moléculaires, ce qui correspond à une succession de sous-bloc similaires à découper pour ensuite les analyser de manière similaire.

## Analyse du fichier pour récupérer les symétries

Pour les symétries, on va avoir à analyser les lignes 896 à 904 dans le fichier d'exemple. Pour voir leur contenu, il est possible d'utiliser deux utilitaires : soit `sed`, soit `awk`.
 1. Afficher les lignes pour voir ce à quoi elles peuvent ressembler. Pour cela on peut soit :
  * utiliser `sed` : il faut spécifier les lignes que l'on veut étudier avec les commandes `sed -n 'XX,YYp' file` pour afficher les lignes comprises entre `XX` et `YY` du fichier `file`
  * utiliser `awk` : idem, il faut spécifier les lignes avec la commande : `awk 'NR>=XX && NR<=YY' file`
Le résultat donne :
```
 Orbital symmetries:
       Occupied  (A1) (E) (T2) (A) (A) (A) (A) (A) (A) (A) (A) (A)
                 (A2) (A) (B2) (A) (A) (B) (A) (A) (A) (A) (A) (A)
                 (A1) (A) (T1) (A) (A) (A) (A) (A) (A) (A) (A) (A)
                 (A) (A) (A) (A) (B1) (A) (A) (A)
       Virtual   (A2) (A1) (A2) (A) (A) (A) (A) (A) (A) (A) (A) (A)
                 (A) (A) (A) (A) (A) (A) (A) (A) (E) (A) (A) (A)
                 (A) (A) (A) (A) (A) (A) (A) (A) (A) (E1) (A) (A)
 The electronic state is 1-A.
```

On peut ainsi voir que ce bloc est délimité par des mots-clé : 
* `Orbital symmetries:` pour le début du bloc de texte
* ` The electronic state is` pour la fin du bloc de texte
* puis que chaque étiquette de symétrie est données par une succession de une lettre et éventuellement un chiffre entre parenthèses.

Comme stratégie, on va isoler le bloc de texte correspondant. Et on va faire en sorte que le script soit auto-adaptatif pour chaque fichier afin de pouvoir facilement automatiser le traitement. On va donc avoir à faire deux étapes :

a. Isoler le bloc de texte qui nous intéresse

b. Faire du traitement sur le bloc isolé pour extraire les informations pertinentes


### Isoler un bloc de texte
Pour isoler le bloc de texte, on va détecter les lignes qui présentent les mots-clé indiquant le début et la fin du bloc.

1. Rédiger une première fonction `findLineKeywords` qui détecte un ensemble de mots clés présents sur des lignes. Faire en sorte que la fontion accepte une liste de mots-clé et retourne un tableau avec :
   
* un tableau indiquant la position des lignes contenant chacun des mots-clé, par défaut, on retourne la dernière occurence
* le nombre total de lignes
  
De manière générale, on simule l'action de la ligne de command `grep -n` en essayant de trouver des lignes particulières. 

### Analyse du bloc
#### Découper le fichier en bloc pour lui appliquer des traitements spécifiques
On va ici découper le bloc de texte qui nous intéresse pour l'isoler. Cela permet de :
* vérifier que le bloc de texte sur lequel on travaille est bien le bon
* pouvoir éventuellement lire uniquement le bloc de texte qui nous intéresse plutôt que de recommencer à lire tout le fichier de départ. C'est intéressant surtout si le fichier de départ est très volumineux.
* On peut bien évidemment supprimer les fichiers annexes créés à la fin si on veut qu'ils soient temporaires.

2. Créer une fonction `cutFile` capable de découper le fichier `infile` initial entre les lignes XX et YY en un plus petit fichier `outfile` à l'aide de la ligne de commande. Pour cela, on pourra utiliser la librairie `subprocess`, en particulier `subprocess.run` qui permet d'exécuter une ligne de commande depuis python comme si on était dans le terminal. Pour utiliser la redirection de la sortie, il est possible d'utiliser les options `shell=True, check=True`. Pour les commandes, on peut utiliser `sed` ou `awk` (voir ci-dessus).


3. Ré-écrire la même fonction en python `cutFilePython` , cela peut permettre de ne pas être dépendant des utilitaires `sed` et `awk` et dépendre du système d'exploitation. Faire attention à ne lire que le minimum de ligne pour être le plus efficace possible si jamais le fichier contient beaucoup de lignes.

#### Traitement du fichier pour extraire les données

Maintenant que l'on a découpé le fichier, il faut récupérer toutes les étiquettes de symétrie. On pourrait essayer d'utiliser `np.genfromtxt`, mais ici, le problème est que les données ne sont pas uniformes et le résultat devrait être un tableau unidimensionnel. Ici, les données sont données sous forme bidimensionnelle et le nombre de colonnes n'est pas identique sur toutes les lignes. De plus, toutes les colonnes ne font pas forcément toutes de la même taille, donc on ne peut pas découper en se basant sur des colonnes de largeur fixe. De plus, il y a les mots `Occupied` et `Virtual` uniquement sur certaines lignes ce qui fait que deux lignes ont un comportement particulier. C'est l'occasion idéale ... d'utiliser des expressions régulières qui sont extrêmement souples !

4. À l'aide d'une expression régulière, écrire une fonction capable d'extraire toutes les symétries listées entre parenthèses dans un fichier et qui retourne une liste contenant toutes les étiquettes de symétrie.

# Extraction des données pour extraire des coefficients
Le fichier présente ensuite des données qui donnent la décomposition d'une orbitale moléculaire sur des orbitales atomiques. On va chercher à récupérer les données réparties sur un grand nombre de lignes pour finir par avoir un tableau des coefficients pour toutes les orbitales.

5. Regarder la structure des sous-blocs pour voir le travail à faire. Pour cela, il faut afficher les lignes 922 à 1005. (avec `sed` ou `awk`, cf ci-dessus)

La structure de chaque sous-bloc est celle donnée sur la figure ci-dessous :
* on a 3 lignes d'en tête, puis `N` coefficients
* on a une colonne d'en-tête puis entre 1 et 5 colonnes qui donnent chacune la projection d'une orbitale moléculaire sur les `N` orbitales atomiques.
![pics/struc-sous-bloc.svg](pics/struc-sous-bloc.svg)

6. Utiliser la fonction `findLineKeywords` pour trouver les indices de début et de fin pour le bloc donnant les orbitales moléculaires

*Ici, on multiplie les lectures du fichier de départ, ce n'est pas optimal, mais on pourrait bien évidemment chercher en une seule fois tous les mots-clé et en profiter pour lire le nombre d'orbitales moléculaires à lire.*

7. Écrire une fonction capable d'aller chercher le nombre `N` d'orbitales atomiques pour savoir le nombre de lignes à lire. Bien faire attention à stocker la valeur sous forme d'entier pour la suite, utiliser une expression régulière.
*Cette information est fournie sous diverses formes dans différentes lignes* (c'est le `80` qui nous intéresse ici) :
```
 There are    80 symmetry adapted cartesian basis functions of A   symmetry.
 There are    80 symmetry adapted basis functions of A   symmetry.
    80 basis functions,   480 primitive gaussians,    80 cartesian basis functions
```
ou
```
 NBasis=    80 RedAO= F EigKep=  0.00D+00  NBF=    80
 NBsUse=    80 1.00D-04 EigRej=  0.00D+00 NBFU=    80
```

On va maintenant découper le fichier en plus petits fichiers pour ensuite pouvoir automatiser le traitement de chacun des sous fichiers. On pourrait tout faire en une seule étape, mais cela complexifierait le code dans son ensemble. Ici, je préfère répéter une succession d'actions simples plutôt qu'un unique traitement plus complexe. Pour cela, on va ré-utiliser la fonction déjà créée auparavant pour découper les fichiers.


8. Découper le fichier de départ en chacun des sous-blocs, on pourra pour cela utiliser la fonction `cutFile` ainsi que les résultats de la question 6. 

Une fois les fichiers découpés, il va être possible d'automatiser le traitement. On peut être plus ou moins souple pour automatiser les choses. Ici, vu que le fichier est relativement bien formaté, on va pouvoir adapter différentes méthodes de traitement. Il serait également possible d'utiliser `awk` pour préformater les données, mais ici, on va appliquer différentes méthodes pour faire des choses plus ou moins compliquées.

## Aller au plus simple avec np.genfromtxt pour lire les coefficients

Si on suppose que les données sont très bien formatées, il est possible de faire en sorte de donner des positions de début et de fin des colonnes. Pour cela, on va spécifier la longueur des colonnes. À la fin, il se peut que le nombre de colonnes ne soit pas égal à 5 dans le cas le plus général. Il faut donc enlever toutes les colonnes qui ne contiennent que des `nan` : il y aura forcément la première qui contient les labels des orbitales atomiques, et peut être des colonnes manquantes à la fin. Il reste ensuite à concaténer tous les coefficients  correctement.

9. Utiliser `np.genfromtxt`, utiliser l'option  `delimiter` avec une suite d'entier pour indiquer la longueur des colonnes à lire `[23,10,10,10,10,10]`. Les trois premières lignes n'étant "pas intéressantes" ici, il est possible de ne pas les lire. De même, la première colonne n'est pas intéressante pour tout de suite. *Il aurait aussi été possible de faire un parser avec des expressions régulières, mais cela aurait été plus long à programmer et à débugguer, j'ai préféré ici privilégier une fonction toute faite relativement robuste*

## Lire les étiquettes des orbitales atomiques
Pour les étiquettes des orbitales atomiques, on a une succession de 4 labels :
* un numéro unique associé à l'orbitale atomique qui va de 1 à 80
* le numéro de l'atome qui porte l'orbitale atomique
* le type d'atome qui porte l'orbitale atomique (carbone, hydrogène, etc)
* le type d'orbitale correspondant : 1S, 2PX, etc


10. Lire les étiquettes dans un des fichiers correspondant aux différents sous-blocs avec `np.genfromtxt`. On pourra utiliser `delimiter` ou tout autre  moyen. On peut également chercher à appliquer un filtre via l'option `converters` pour supprimer tous les espaces inutiles en début et en fin de chaîne de caractère.

On a récupéré les colonnes, mais il y a des données manquantes, par exemple, pour l'orbitale 3, il manque le numéro d'atome et le type d'atome. 

11. Écrire une portion de code capable de compléter les colonnes vides avec les dernières valeurs non vides dans la ligne au dessus.

## Un exemple d'utilisation du travail effectué

On peut maintenant faire des choses sympatiques comme afficher tous les coefficients importants pour chaque orbitale moléculaire.



### Quelques alternatives

#### Utilisation native de grep ou awk pour détecter les lignes
Pour trouver les lignes contenant les mots-clés, on peut utiliser `grep -n` et pour compter les lignes, on peut utiliser la commande `grep -c ^ file`. Il est aussi possible d'utiliser `awk`.

## Une autre méthode sans découper en sous-fichiers pour lire les coefficients

On peut également faire un essai en faisant un unique np.genfromtxt depuis le fichier de départ, mais cela demande plus de réflexion. En effet, on doit inclure les lignes d'en tête, puis les supprimer et faire un reformatage intelligent du tableau avec une combinaison de `split` et `stack`. Par contre, cela donne assez simplement accès aux valeurs propres associées à chaque orbitale.